In [162]:
import pandas as pd
import nibabel as nib
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [49]:
PATH_images = "/Users/theomathieu/Documents/Stage/results_img/cadotte/new-version-LPI/LPI"
PATH_labels = "/Users/theomathieu/Documents/Stage/results_img/cadotte/new-version-LPI/extracted_label"

In [233]:
subjects = []
for file in os.listdir(PATH_images):
    basename = "_".join(file.split("_")[:2])
    if basename not in subjects:
        subjects.append(basename)


def get_all_files(subject):
    sub_name = subject.split("_")[0]
    sub_name = "sub-" + sub_name.split("-")[-1]
    centerline_path = os.path.join(PATH_images, subject + "_0000_centerline.csv")
    sc_path = os.path.join(PATH_images, subject + "_0000_seg.nii.gz")
    pmj_path = os.path.join(PATH_labels, sub_name + "_T2w_label_pmj.nii.gz")
    path_rootlet = os.path.join(PATH_labels, sub_name + "_T2w_label_rootlet-bin.nii.gz")
    path_vertebrae = os.path.join(PATH_labels, sub_name + "_T2w_label_vertebrae.nii.gz")
    return [centerline_path, sc_path, pmj_path, path_rootlet, path_vertebrae]


def group_consecutive_elements(input_list):
    grouped_list = []
    min_max_dir = {}
    sublist = []

    for i, num in enumerate(input_list):
        if i == 0 or num - 1 == input_list[i - 1] or num - 2 == input_list[i - 1]:
            sublist.append(num)
        else:
            grouped_list.append(sublist)
            sublist = [num]

    if sublist:
        grouped_list.append(sublist)

    for i, part in enumerate(grouped_list):
        min_max_dir[len(grouped_list) - i] = [min(part), max(part), np.mean(part)]

    return min_max_dir, grouped_list


def get_distance_from_pmj(centerline_points, z_index, px, py, pz):
    """

    """
    length = 0
    arr_length = [0]
    for i in range(z_index, 0, -1):
        distance = np.sqrt(((centerline_points[i, 0] - centerline_points[i - 1, 0]) * px) ** 2 +
                           ((centerline_points[i, 1] - centerline_points[i - 1, 1]) * py) ** 2 +
                           ((centerline_points[i, 2] - centerline_points[i - 1, 2]) * pz) ** 2)
        length += distance
        arr_length.append(length)
    arr_length = arr_length[::-1]
    arr_length = np.stack((arr_length, centerline_points[:z_index + 1, 2]), axis=0)
    return arr_length


def get_zoom(path):
    img = nib.load(path)
    hdr = img.header
    px = hdr.get_zooms()[0]
    py = hdr.get_zooms()[1]
    pz = hdr.get_zooms()[2]
    return px, py, pz


def get_slices(path):
    img = nib.load(path)
    all = img.get_fdata()
    slices = np.unique(np.where(all > 0)[2])
    return slices


def get_pmj_position(path_pmj):
    """
    Get the position of the pmj in the image in voxel
    Args:
        path_pmj (str): Path to the pmj segmentation
    Returns:
        pmj_position (int): Position of the pmj in the image in voxel
    """
    pmj_img = nib.load(path_pmj).get_fdata()
    pmj_position = np.where(pmj_img > 0)
    return pmj_position


In [253]:
print(subjects)
df_root = {"sub_name": [], "level": [], "PMJ_start": [], "PMJ_end": [], "slice_s":[], "slice_e":[]}
df_ver = {"sub_name": [], "level": [], "PMJ_start": [], "PMJ_end": [], "slice_s":[], "slice_e":[]}

for subject in subjects:
    print(subject)
    centerline_path, sc_path, pmj_path, path_rootlet, path_vertebrae = get_all_files(subject)
    px, py, pz = get_zoom(sc_path)
    root_grouped, _ = group_consecutive_elements(get_slices(path_rootlet))
    vert_grouped, _ = group_consecutive_elements(get_slices(path_vertebrae))
    pmj = get_pmj_position(pmj_path)
    z_index = min(pmj[2])
    print(z_index)
    centerline_points = np.genfromtxt(centerline_path,
                                      delimiter=',')
    arr_length = get_distance_from_pmj(centerline_points, z_index, px, py, pz)
    for i, level in enumerate(root_grouped):
        value = root_grouped[level]
        df_root["sub_name"].append(subject)
        df_root["level"].append(level + 2)
        df_root["PMJ_start"].append(-arr_length[0][value[1]])
        df_root["PMJ_end"].append(-arr_length[0][value[0]])
        df_root["slice_s"].append(value[1])
        df_root["slice_e"].append(value[0])

    level_count = 1
    for vertebrae in range(1, len(vert_grouped) // 2 + 1):
        start = vertebrae * 2 - 1
        end = vertebrae * 2
        df_ver["sub_name"].append(subject)
        df_ver["level"].append(level_count)
        df_ver["slice_s"].append(vert_grouped[start][1])
        df_ver["slice_e"].append(vert_grouped[end][0])
        df_ver["PMJ_start"].append(-arr_length[0][vert_grouped[start][1]])
        df_ver["PMJ_end"].append(-arr_length[0][vert_grouped[end][0]])
        level_count += 1

df_root_pd = pd.DataFrame(df_root)
df_ver_pd = pd.DataFrame(df_ver)

['sub-5-13755_009', 'sub-11-14411_013', 'sub-8-14350_011', 'sub-7-14348_003', 'sub-4-13747_010', 'sub-3-13746_012', 'sub-13-14493_008', 'sub-6-13757_004', 'sub-12-14492_007', 'sub-9-14406_001', 'sub-10-14407_000', 'sub-14-14693_002', 'sub-1-13696_006', 'sub-2-13697_005']
sub-5-13755_009
467
sub-11-14411_013
474
sub-8-14350_011
489
sub-7-14348_003
475
sub-4-13747_010
476
sub-3-13746_012
472
sub-13-14493_008
484
sub-6-13757_004
469
sub-12-14492_007
483
sub-9-14406_001
498
sub-10-14407_000
495
sub-14-14693_002
506
sub-1-13696_006
487
sub-2-13697_005
452


In [256]:
df_root_pd.to_csv("/Users/theomathieu/Documents/Stage/Code/model-spinal-rootlets/dataset_creation/cadotte_rootlet_dist.csv", index=False)
df_ver_pd.to_csv("/Users/theomathieu/Documents/Stage/Code/model-spinal-rootlets/cadotte_vertebrae_dist.csv", index= False)

In [238]:
subject = 1
#colors = ['#5CC878', '#FF707D', '#64DEDC', '#D682C6', '#E1E074', '#738282', '#C16200', '#197278', '#EF9CDA']
colors = {2: '#5B6C5D', 3: '#5CC878', 4: '#FF707D', 5: '#64DEDC', 6: '#D682C6', 7: '#E1E074', 8: '#738282', 9: '#C16200', 10: '#197278', 11: '#EF9CDA'}

fig = go.Figure()
print(f"what\tLevel\tstart\tend")
for i, subject in enumerate(df_root_pd["sub_name"].unique()):
    df_root_sub = df_root_pd[df_root_pd["sub_name"] == subject]
    df_ver_sub = df_ver_pd[df_ver_pd["sub_name"] == subject]
    id = int(subject.split("-")[1])
    print(subject, id)
    for idx, level in enumerate(df_root_sub["level"].unique()):
        df_level_pmj = df_root_sub[df_root_sub["level"] == level]
        if not np.isnan(df_level_pmj["PMJ_start"].values[0]):
            root_s = df_level_pmj["PMJ_start"].values[0]
            root_e = df_level_pmj["PMJ_end"].values[0]
            fig.add_shape(
                type="rect",
                x0=id - 0.175, y0=root_e, x1=id - 0.425, y1=root_s,
                line=dict(color=colors[level], width=2),
                fillcolor=colors[level],
                name=f"Predicted {level}",
            )
        else:
            root_s = np.nan
            root_e = np.nan
        print(f"root\t{level}\t{root_s:.2f}/{root_e:.2f}")

    for idx, level in enumerate(df_ver_sub["level"].unique()[1:]):
        df_level_cad = df_ver_sub[df_ver_sub["level"] == level]
        if not np.isnan(df_level_cad["PMJ_start"].values[0]):
            ver_s = df_level_cad["PMJ_start"].values[0]
            ver_e = df_level_cad["PMJ_end"].values[0]
            fig.add_shape(
                type="rect",
                x0=id + 0.125, y0=ver_s, x1=id - 0.125, y1=ver_e,
                line=dict(color="grey", width=1),
                fillcolor="#F4F4CC",
                name=f"Cadotte {level}"
            )
        else:
            ver_s = np.nan
            ver_e = np.nan
        print(f"vertebrae\t{level}\t{ver_s:.2f}/{ver_e:.2f}")

fig.update_layout(
    title="Nerve rootlet layout",
    xaxis_title="Subject",
    yaxis_title="PMJ distance (PMJ = 0 mm)",
    xaxis_range=[0, 15],  # Adjust the x-axis range as needed
    yaxis_range=[min(df_root_pd["PMJ_end"]) - 5, max(df_root_pd["PMJ_start"])+5],  # Adjust the y-axis range as needed
    yaxis_autorange='reversed',
    showlegend=True,
    xaxis=dict(ticktext=[str(x) for x in range(1, 15)], tickvals=[*range(1, 15)]),
    yaxis=dict(dtick=10)
)
fig.show()

what	Level	start	end
sub-5-13755_009 5
root	9	-136.92/-145.96
root	8	-122.71/-132.22
root	7	-111.82/-120.37
root	6	-98.63/-106.83
root	5	-83.46/-93.62
root	4	-66.08/-75.57
root	3	-51.12/-61.19
vertebrae	2	-69.80/-85.81
vertebrae	3	-96.90/-116.43
vertebrae	4	-126.92/-132.22
sub-11-14411_013 11
root	9	-131.96/-141.96
root	8	-120.10/-127.61
root	7	-107.11/-115.48
root	6	-92.74/-103.21
root	5	-78.29/-88.06
root	4	-61.29/-72.04
root	3	-46.26/-55.72
vertebrae	2	-58.17/-74.39
vertebrae	3	-75.56/-93.92
vertebrae	4	-105.55/-126.63
sub-8-14350_011 8
root	9	-146.62/-154.76
root	8	-128.39/-136.59
root	7	-111.37/-120.17
root	6	-95.04/-102.27
root	5	-78.39/-88.87
root	4	-64.09/-71.34
root	3	-46.82/-57.72
vertebrae	2	-64.09/-79.77
vertebrae	3	-83.39/-97.38
vertebrae	4	-100.89/-116.06
vertebrae	5	-119.68/-135.81
vertebrae	6	-136.98/-153.97
sub-7-14348_003 7
root	9	-132.51/-143.91
root	8	-122.12/-129.85
root	7	-110.01/-118.21
root	6	-95.63/-105.32
root	5	-83.98/-90.93
root	4	-66.40/-78.29
root	3	-49.11

In [232]:
df_root_pd

,sub_name,level,PMJ_start,PMJ_end,slice_s,slice_e
0,sub-5-13755_009,9,-136.924703,-145.959067,134,114
1,sub-5-13755_009,8,-122.710674,-132.220648,167,145
2,sub-5-13755_009,7,-111.824036,-120.367074,193,173
3,sub-5-13755_009,6,-98.628645,-106.831245,226,205
4,sub-5-13755_009,5,-83.462261,-93.617861,264,238
...,...,...,...,...,...,...
96,sub-2-13697_005,7,-112.715285,-113.598370,191,189
97,sub-2-13697_005,6,-101.387886,-109.199885,220,200
98,sub-2-13697_005,5,-85.831895,-93.660833,258,239
99,sub-2-13697_005,4,-67.660781,-77.085995,298,277


In [213]:
df_ver_pd

,sub_name,level,PMJ_start,PMJ_end,slice_s,slice_e
0,sub-5-13755_009,1,-5.485255,-67.354024,454,303
1,sub-5-13755_009,2,-69.799487,-83.462261,297,264
2,sub-5-13755_009,3,-84.243461,-98.628645,262,226
3,sub-5-13755_009,4,-99.409845,-113.386436,224,189
4,sub-5-13755_009,5,-114.167636,-128.484665,187,153
...,...,...,...,...,...,...
72,sub-2-13697_005,2,-62.158504,-78.070965,310,275
73,sub-2-13697_005,3,-79.837134,-97.481886,271,230
74,sub-2-13697_005,4,-100.606686,-115.466423,222,185
75,sub-2-13697_005,5,-117.419423,-131.277991,180,149
